# CIAN.ru

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [24]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [11]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

In [296]:
def getDist(flat_page):
    centre = [55.751999, 37.617734]
    x_dist = (getCoords(flat_page)[0] - centre[0])*111 #111 километров между двумя градусами широты
    y_dist = (getCoords(flat_page)[1] - centre[1])*70 # на данной широте расстояние между двумя градусами долготы примерно 70 километров
    dist = math.sqrt(pow(x_dist,2) + pow(y_dist,2))
    return dist


56.48403748867993


In [300]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    if room_number == '':  #была какая-та квартира без указания количества комнат
        room_number = 'None'
    return room_number

In [236]:
def getNfloors(table):
    floor = re.split('Этаж:|Тип дома', table)[1] # знаме два слова, между которыми значение этажа, следовательно, при разбиении берем второй элемент, по такому принципу будем брать все необходимые переменные
    house_floor = re.split('/',floor)
    if len(house_floor)==2:
        house_floor = re.split('/',floor)[1]
        house_floor = re.split(' |\n|\xa0', house_floor)
        for i in house_floor:
            if i.isdigit():  
                house_floor = i
    else:
        house_floor = 'None'
    return house_floor
    


In [234]:
def getFloor(table):
    floor = re.split('Этаж:|Тип дома', table)[1]
    flat_floor = re.split('/',floor)[0]
    flat_floor = re.split(' |\n|\xa0', flat_floor)
    for i in flat_floor:
        if i.isdigit():  
            flat_floor = i
    return flat_floor
    

In [241]:
def getTotsp(table):
    tot_sp = re.split('Общая площадь:|м2', table)[1] #знаем, что точно находится между Общей площадью и первым м2
    tot_sp = tot_sp.replace(',','.') # представим вещественные числа в привычном виде(с точкой)
    tot_sp = re.split(' |\n|\xa0', tot_sp) #вроде бы только эти символы могут попадаться 
    for i in tot_sp:
        string = str(i)
        if string.replace(".","1").isdigit():  # вещественные числа не пройдут через isdigit, следовательно, на время превратим его в любое целое
            tot_sp = i
    return tot_sp 

In [237]:
def getLivesp(table):
    live_sp = re.split('Жилая площадь:|Площадь кухни', table)
    live_sp = re.split('м2', live_sp[1])
    if len(live_sp) == 2:
        live_sp = live_sp[0]
        live_sp = live_sp.replace(',','.')
        live_sp = re.split(' |\n|\xa0', live_sp)
        for i in live_sp:
            string = str(i)
            if string.replace(".","1").isdigit():  # вещественные числа не пройдут через isdigit, следовательно, на время превратим его в любое целое
                live_sp = i
    else:
        live_sp = 'None'
    return live_sp

In [238]:
kit_sp = re.split('Площадь кухни:', table)[1]
kit_sp = re.split('м2', kit_sp)
print(kit_sp[0])
def getKitsp(table):
    kit_sp = re.split('Площадь кухни:', table)[1]
    kit_sp = re.split('м2', kit_sp)# далее ничего в м2 нет
    if len(kit_sp) == 2:
        kit_sp = kit_sp[0] 
        kit_sp = kit_sp.replace(',','.')
        kit_sp = re.split(' |\n|\xa0', kit_sp)
        for i in kit_sp:
            string = str(i)
            if string.replace(".","1").isdigit():  # вещественные числа не пройдут через isdigit, следовательно, на время превратим его в любое целое
                kit_sp = i
    else:
        kit_sp = 'None'
    return kit_sp



6 


In [242]:
def getMetrdist(flat_page):
    Metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    Metrdist = html_stripper(Metrdist)
    
    if Metrdist=='None':         #хотел показать, что бывают квартиры и без сведений о ближайшем метро
        Metrdist = 'None'
    else:
        Metrdist = re.split('мин.', Metrdist)[0]
        Metrdist = re.split(' |\n|\xa0', Metrdist)
        for i in Metrdist:
            if i.isdigit():  
                Metrdist = i
    return Metrdist

In [129]:
def getWalk(flat_page):
    Walk = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    Walk = html_stripper(Walk)
    if Walk=='None':                         
        Walk = 'None'
    elif len(re.split('пешком', Walk))==2:   #если это слово встретилось, то таблица разобьется на две части
        Walk = 1
    else:
        Walk = 0
    return Walk

In [63]:
def getBrick(table):
    if len(re.split('монолитный|кирпичный|железо-бетонный', table))==2: 
        Brick = 1
    else:
        Brick = 0
    return Brick

In [57]:
def getBal(table):
    Bal = re.split('Балкон:|Лифт', table)[1]
    if len(re.split('1|2|3|4|5',Bal)) > 1:  #вряд ли есть квартиры с количеством балконов больше 5
        Bal = 1
    else:
        Bal = 0
    return Bal

In [135]:
def getTel(table):
    Tel = re.split('Телефон', table)
    if len(Tel) == 2:
        Tel = re.split('Телефон', table)[1]
        Tel = re.split('да',Tel) # после телефона слово "да" больше не встречается
        if len(Tel)==2:
            Tel = 1
        else:
            Tel = 0
    else:
        Tel = 'None'  #невсегда это поле присутствует в таблице, то есть, иногда ничего поповоду телефона не можем сказать
    return Tel

In [59]:
def getNew(table):
    if len(re.split('вторичка', table))==2: 
        New = 0
    else:
        New = 1
    return New

In [295]:
district_list=[] #тут храним ссылки на разные округи
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B1%5D=24&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B1%5D=40&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B1%5D=57&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B1%5D=73&district%5B10%5D=82&district%5B11%5D=83&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B1%5D=85&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B1%5D=101&district%5B10%5D=110&district%5B11%5D=111&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B1%5D=113&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B1%5D=328&district%5B10%5D=337&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district_list.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&p=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
flatStats = pd.DataFrame(columns=['N','Price', 'Dist', 'Rooms','Nfloors','Floor','Totsp','Livesp','Kitsp','Metrdist','Walk','Brick','Bal','Tel','New'])
links = []
for district in district_list:
    for page in range(1, 11):  # не успел сделать для 30 страниц
        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)

In [297]:
counter = 0; # считает номер квартиры
for link in links:
    counter = counter+1
    flat_url = 'http://www.cian.ru/sale/flat/' + str(link) + '/'
    flat_page = requests.get(flat_url)
    flat_page = flat_page.content
    flat_page = BeautifulSoup(flat_page, 'lxml')
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    to_append = {'N' : counter, 'Price':getPrice(flat_page), 'Dist':getDist(flat_page), 'Rooms':getRoom(flat_page), 'Nfloors':getNfloors(table), 'Floor':getFloor(table), 'Totsp': getTotsp(table), 'Livesp' : getLivesp(table), 'Kitsp' : getKitsp(table),'Metrdist' : getMetrdist(flat_page), 'Walk' : getWalk(flat_page), 'Brick' : getBrick(table), 'Bal' : getBal(table), 'Tel' : getTel(table), 'New' :getNew(table) }
    flatStats = flatStats.append(to_append, ignore_index=True)
    print('I`m not dead, I`m working! The link is {}'.format(link))
   

I`m not dead, I`m working! The link is 149724237
I`m not dead, I`m working! The link is 150882335
I`m not dead, I`m working! The link is 150012891
I`m not dead, I`m working! The link is 151004941
I`m not dead, I`m working! The link is 150346368
I`m not dead, I`m working! The link is 150640194
I`m not dead, I`m working! The link is 149031321
I`m not dead, I`m working! The link is 151004981
I`m not dead, I`m working! The link is 148794719
I`m not dead, I`m working! The link is 149565205
I`m not dead, I`m working! The link is 149849414
I`m not dead, I`m working! The link is 149300855
I`m not dead, I`m working! The link is 149788538
I`m not dead, I`m working! The link is 148112883
I`m not dead, I`m working! The link is 149727197
I`m not dead, I`m working! The link is 149031382
I`m not dead, I`m working! The link is 151005044
I`m not dead, I`m working! The link is 150910021
I`m not dead, I`m working! The link is 151005116
I`m not dead, I`m working! The link is 148800467
I`m not dead, I`m wo

In [298]:
flatStats.tail()

,N,Price,Dist,Rooms,Nfloors,Floor,Totsp,Livesp,Kitsp,Metrdist,Walk,Brick,Bal,Tel,New
3075,3076.0,3850000.0,51.104046,3,5,3,63,47,6,None,None,1.0,1.0,None,0.0
3076,3077.0,3850000.0,56.761953,2,5,5,43,29,6,None,None,0.0,1.0,None,0.0
3077,3078.0,3850000.0,63.538991,1,16,7,48.7,22,10,30,0,0.0,1.0,0,0.0
3078,3079.0,3900000.0,56.484037,3,None,1,69.4,47,9.6,None,None,0.0,1.0,None,1.0
3079,3080.0,3999000.0,56.484037,3,None,3,74,49.3,8.3,None,None,0.0,1.0,None,1.0


In [299]:
flatStats.to_csv('C:/Users/18/Documents/CMF/Parsing_Web_Data.csv',index=False)